# From twitter

Obtener tweets relacionados con el [sismo](http://ciencia.unam.mx/leer/652/-que-ocurrio-el-19-de-septiembre-de-2017-en-mexico-) vivido en la Ciudad de México en la fecha de $2029/\text{Sept}/19$.

In [1]:
import searchtweets
import pandas as pd
import numpy as np

Verificar que la versión del paquete [`searchtweets`](https://pypi.org/project/searchtweets-v2/) es **1.0.7**. La manera que se recomienda instalar el paquete es la siguiente:
+ `pip install searchtweets-v2`

In [2]:
searchtweets.__version__

'1.0.7'

In [3]:
from searchtweets import ResultStream, load_credentials, gen_request_parameters, collect_results

In [4]:
help(gen_request_parameters)

Help on function gen_request_parameters in module searchtweets.api_utils:

gen_request_parameters(query, results_per_call=None, start_time=None, end_time=None, since_id=None, until_id=None, tweet_fields=None, user_fields=None, media_fields=None, place_fields=None, poll_fields=None, expansions=None, stringify=True)
    Generates the dict or json payload for a search query.
    
    Args:
        query (str): The string version of a search query,
            e.g., "snow has:media -is:retweet". Accepts multi-line strings
            for ease of entry.
        results_per_call (int): number of tweets or counts returned per API
        call. This maps to the `max_results`` search API parameter.
            Defaults to 100 (maximum supported in Labs).
        start_time (str or None): Date format as specified by
            `convert_utc_time` for the starting time of your search.
        end_time (str or None): date format as specified by `convert_utc_time`
            for the end time of yo

Verificar que se tiene el archivo `conf/local/credentials.yaml` con las credenciales necesarias para realizar el *scrapping* correspondiente a este notebook. En caso de que no se tengan se recomienda a acudir con los autores del proyecto.

In [5]:
search_args = load_credentials(filename="../conf/local/credentials.yaml",
                               yaml_key="search_tweets_v2",
                               env_overwrite=False)

In [6]:
query = gen_request_parameters("sismo cdmx", 
                               start_time = "2017-09-11",
                               end_time = "2017-09-13",
                               tweet_fields="id,created_at,conversation_id,in_reply_to_user_id,referenced_tweets,entities,text",
                               user_fields="id,username",
                               expansions="author_id",
                               results_per_call=10)

In [7]:
tweets = collect_results(query,
                         max_tweets=1,
                         result_stream_args=search_args) # change this if you need to

In [8]:
rs = ResultStream(request_parameters=query,
                  max_results=10,
                  max_pages=1,
                  **search_args)
print(rs)

ResultStream: 
	{
    "endpoint": "https://api.twitter.com/2/tweets/search/all",
    "request_parameters": {
        "query": "sismo cdmx",
        "max_results": 10,
        "start_time": "2017-09-11T00:00:00Z",
        "end_time": "2017-09-13T00:00:00Z",
        "tweet.fields": "id,created_at,conversation_id,in_reply_to_user_id,referenced_tweets,entities,text",
        "user.fields": "id,username",
        "expansions": "author_id"
    },
    "max_tweets": 500
}


In [9]:
results = list(rs.stream())

In [10]:
res = pd.DataFrame(results)

In [11]:
res.head(3)

,data,includes,meta
0,[{'text': '#LoMásLeído Se registra el sismo má...,"{'users': [{'id': '1380764066', 'name': 'Capit...","{'newest_id': '907755462241775617', 'oldest_id..."
1,[{'text': '#LoMásLeído Se desploma avión de la...,"{'users': [{'id': '1380764066', 'name': 'Capit...","{'newest_id': '907750426937888768', 'oldest_id..."
2,[{'text': 'RT @CapitalMX_: #LoMásLeído Propone...,"{'users': [{'id': '108482463', 'name': 'Luis E...","{'newest_id': '907749085465251840', 'oldest_id..."


In [12]:
res.shape

(54, 3)

In [13]:
res.data[0]

[{'text': '#LoMásLeído Se registra el sismo más fuerte en 100 años en México; reportan cinco muertos https://t.co/SQzgN5sjgJ',
  'created_at': '2017-09-12T23:59:02.000Z',
  'author_id': '1380764066',
  'entities': {'annotations': [{'start': 59,
     'end': 64,
     'probability': 0.9962,
     'type': 'Place',
     'normalized_text': 'México'}],
   'urls': [{'start': 90,
     'end': 113,
     'url': 'https://t.co/SQzgN5sjgJ',
     'expanded_url': 'http://capital-cdmx.org/nota-Se-registra-el-sismo-mas-fuerte-en-100-anos-en-Mexico--reportan-cinco-muertos-20178952',
     'display_url': 'capital-cdmx.org/nota-Se-regist…'}],
   'hashtags': [{'start': 0, 'end': 11, 'tag': 'LoMásLeído'}]},
  'id': '907755462241775617',
  'conversation_id': '907755462241775617'},
 {'text': '#EntérateCDMX Reubican a 19 planteles de la Capital CDMX afectados por sismo https://t.co/A5eky5Rp5g',
  'created_at': '2017-09-12T23:54:55.000Z',
  'author_id': '1380764066',
  'entities': {'annotations': [{'start': 44,
   

In [14]:
df_1 = pd.DataFrame()
for api in res.data:
    for tweet in api:
        if 'in_reply_to_user_id' not in tweet:
            tweet['in_reply_to_user_id'] = 'propio'
        if 'referenced_tweets' not in tweet:
            tweet['referenced_tweets'] = 'propio_original'
        if 'entities' not in tweet:
            tweet['entities'] = 'propio_original'
        lista = np.array([tweet['created_at'], tweet['id'], tweet['author_id'], 
                          tweet['conversation_id'], tweet['in_reply_to_user_id'], tweet['referenced_tweets'],
                          tweet['entities'],tweet['text']])
        temp = pd.DataFrame([lista], columns=['created_at', 'id', 'author_id', 'conversation_id', 'referenced_tweets',
                                              'reply_to_user_id', 'entities' ,'text'])
        df_1 = pd.concat([df_1,temp], axis=0)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if sys.path[0] == '':


In [15]:
df_1.head(6)

,created_at,id,author_id,conversation_id,referenced_tweets,reply_to_user_id,entities,text
0,2017-09-12T23:59:02.000Z,907755462241775617,1380764066,907755462241775617,propio,propio_original,"{'annotations': [{'start': 59, 'end': 64, 'pro...",#LoMásLeído Se registra el sismo más fuerte en...
0,2017-09-12T23:54:55.000Z,907754428371005441,1380764066,907754428371005441,propio,propio_original,"{'annotations': [{'start': 44, 'end': 55, 'pro...",#EntérateCDMX Reubican a 19 planteles de la Ca...
0,2017-09-12T23:54:19.000Z,907754276415557633,944251843,907754276415557633,propio,propio_original,"{'mentions': [{'start': 8, 'end': 18, 'usernam...",Instala @ISSSTE_mx #CentrosDeAcopio en la #CDM...
0,2017-09-12T23:54:18.000Z,907754271847968768,705194891588964352,907754271847968768,propio,propio_original,"{'mentions': [{'start': 8, 'end': 18, 'usernam...",Instala @ISSSTE_mx #CentrosDeAcopio en la #CDM...
0,2017-09-12T23:53:12.000Z,907753994147254272,2369573840,907753994147254272,propio,"[{'type': 'retweeted', 'id': '9073255084126822...","{'mentions': [{'start': 3, 'end': 9, 'username...",RT @CFEmx: A sus órdenes 7 #CentrosDeAcopio en...
0,2017-09-12T23:46:08.000Z,907752218098626560,766351022159699968,907752218098626560,propio,"[{'type': 'retweeted', 'id': '9077015764202618...","{'mentions': [{'start': 3, 'end': 16, 'usernam...",RT @JuanCLastiri: En las oficinas de @SEDATU_m...


In [16]:
df_2 = pd.DataFrame()
for api in res.includes:
    for tweet in api['users']:    
        lista = np.array([tweet['id'], tweet['username']])
        temp = temp = pd.DataFrame([lista], columns=['author_id', 'username'])
        df_2 = pd.concat([df_2,temp], axis=0)

In [17]:
df_2['id_user'] = df_2.author_id.astype('str') + df_2.username

In [18]:
df_2.head(6)

,author_id,username,id_user
0,1380764066,CapitalMX_,1380764066CapitalMX_
0,944251843,AlexNavaMx,944251843AlexNavaMx
0,705194891588964352,AlternativaGDM,705194891588964352AlternativaGDM
0,2369573840,AIBMontesDeOca,2369573840AIBMontesDeOca
0,766351022159699968,prepSEDATU,766351022159699968prepSEDATU
0,903087297729572864,ClaudiaMunozpaz,903087297729572864ClaudiaMunozpaz


In [19]:
df_2=df_2.drop_duplicates('id_user')

In [20]:
len(pd.unique(df_1.id))

506

In [21]:
df_2.shape

(417, 3)

In [22]:
df_2_clean = df_2[['author_id','username']]

In [23]:
df = df_1.merge(df_2_clean, on='author_id', how='left')

In [24]:
df

,created_at,id,author_id,conversation_id,referenced_tweets,reply_to_user_id,entities,text,username
0,2017-09-12T23:59:02.000Z,907755462241775617,1380764066,907755462241775617,propio,propio_original,"{'annotations': [{'start': 59, 'end': 64, 'pro...",#LoMásLeído Se registra el sismo más fuerte en...,CapitalMX_
1,2017-09-12T23:54:55.000Z,907754428371005441,1380764066,907754428371005441,propio,propio_original,"{'annotations': [{'start': 44, 'end': 55, 'pro...",#EntérateCDMX Reubican a 19 planteles de la Ca...,CapitalMX_
2,2017-09-12T23:54:19.000Z,907754276415557633,944251843,907754276415557633,propio,propio_original,"{'mentions': [{'start': 8, 'end': 18, 'usernam...",Instala @ISSSTE_mx #CentrosDeAcopio en la #CDM...,AlexNavaMx
3,2017-09-12T23:54:18.000Z,907754271847968768,705194891588964352,907754271847968768,propio,propio_original,"{'mentions': [{'start': 8, 'end': 18, 'usernam...",Instala @ISSSTE_mx #CentrosDeAcopio en la #CDM...,AlternativaGDM
4,2017-09-12T23:53:12.000Z,907753994147254272,2369573840,907753994147254272,propio,"[{'type': 'retweeted', 'id': '9073255084126822...","{'mentions': [{'start': 3, 'end': 9, 'username...",RT @CFEmx: A sus órdenes 7 #CentrosDeAcopio en...,AIBMontesDeOca
...,...,...,...,...,...,...,...,...,...
501,2017-09-12T08:29:38.000Z,907521572419796992,63120239,907521572419796992,propio,propio_original,"{'annotations': [{'start': 81, 'end': 84, 'pro...",#infórmate Reiniciarán clases mañana 5 de 19 e...,netdevelopmx
502,2017-09-12T08:24:05.000Z,907520176228372480,796868290211676161,907520176228372480,propio,propio_original,"{'urls': [{'start': 14, 'end': 37, 'url': 'htt...",El Financiero https://t.co/rsTRj7RPHM,boris_abad
503,2017-09-12T08:06:13.000Z,907515678202220545,478199135,907515678202220545,propio,"[{'type': 'retweeted', 'id': '9060269812919951...","{'urls': [{'start': 67, 'end': 90, 'url': 'htt...",RT @Excelsior: Así se vivió el sismo en el @Me...,diaynoche303
504,2017-09-12T08:03:40.000Z,907515039283806208,798227800612335616,907515039283806208,propio,propio_original,"{'annotations': [{'start': 42, 'end': 45, 'pro...",Reabrirán 5 de las 19 escuelas dañadas en CdMx...,santiago071L


In [25]:
df.dtypes

created_at           object
id                   object
author_id            object
conversation_id      object
referenced_tweets    object
reply_to_user_id     object
entities             object
text                 object
username             object
dtype: object

In [26]:
df.to_csv("sismo_cdmx_506.csv")

In [27]:
df.entities

0      {'annotations': [{'start': 59, 'end': 64, 'pro...
1      {'annotations': [{'start': 44, 'end': 55, 'pro...
2      {'mentions': [{'start': 8, 'end': 18, 'usernam...
3      {'mentions': [{'start': 8, 'end': 18, 'usernam...
4      {'mentions': [{'start': 3, 'end': 9, 'username...
                             ...                        
501    {'annotations': [{'start': 81, 'end': 84, 'pro...
502    {'urls': [{'start': 14, 'end': 37, 'url': 'htt...
503    {'urls': [{'start': 67, 'end': 90, 'url': 'htt...
504    {'annotations': [{'start': 42, 'end': 45, 'pro...
505    {'annotations': [{'start': 58, 'end': 69, 'pro...
Name: entities, Length: 506, dtype: object